In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [2]:
## Load the datasets

max_features = 10000 ## vocabulary size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

print(f'Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape: {X_train.shape}, Testing labels shape: {y_test.shape}')

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training data shape: (25000,), Training labels shape: (25000,)
Testing data shape: (25000,), Testing labels shape: (25000,)


In [24]:
# X_train[0] # onehot representation of each index

In [4]:
### Inspect a sample review and its label
sample_review = X_train[0]
sample_label = y_train[0]

print(f"sample review (as integers):{sample_review}")
print(f"Sample label: {sample_label}")

sample review (as integers):[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample label: 1


In [25]:
## Mapping of word index back to word
word_index=imdb.get_word_index()
# word_index
reverse_word_index = {value: key for key, value in word_index.items()}
# reverse_word_index

In [6]:
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in sample_review])
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [7]:
max_len = 500

X_train=sequence.pad_sequences(X_train, maxlen=max_len)
X_test=sequence.pad_sequences(X_test, maxlen=max_len)
X_train


array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [8]:
# train simple rnn
model=Sequential()
model.add(Embedding(input_dim=max_features, output_dim=128, input_length = max_len)) ## Embedding Layers
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
model.build(input_shape=(None, max_len))


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# create an instance of earlystopping callback

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
early_stopping

In [13]:
## train the model with early stopping
model.fit(
    X_train, y_train,
    epochs=10, batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 225ms/step - accuracy: 0.5576 - loss: 64753463296.0000 - val_accuracy: 0.5902 - val_loss: 0.6660
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 141s 223ms/step - accuracy: 0.6569 - loss: 0.6463 - val_accuracy: 0.6366 - val_loss: 0.6389
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 141s 226ms/step - accuracy: 0.7170 - loss: 0.6053 - val_accuracy: 0.6472 - val_loss: 0.6182
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 228ms/step - accuracy: 0.7431 - loss: 0.5521 - val_accuracy: 0.6538 - val_loss: 0.6071
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 202s 229ms/step - accuracy: 0.7752 - loss: 0.4854 - val_accuracy: 0.6776 - val_loss: 0.5914
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 195s 218ms/step - accuracy: 0.8111 - loss: 0.4194 - val_accuracy: 0.6870 - val_loss: 0.5982
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 147s 226ms/step - accuracy: 0.8340 - loss: 0.3764 - val_accuracy: 0.6966 - val_loss: 0.6117
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 223ms/step - accuracy

In [14]:
## save the model
model.save('simple_rnn_imdb.h5')

In [15]:
### prediction
from tensorflow.keras.models import load_model
model = load_model('simple_rnn_imdb.h5')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [16]:
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key,value in word_index.items()}

In [17]:
def decode_review(encoded_review):
  return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

def preprocess_text(text):
  words = text.lower().split()
  encoded_review = [word_index.get(word, 2) + 3 for word in words]
  padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
  return padded_review

In [26]:
### prediction function

def predict_sentiment(review):
  preprocessed_input=preprocess_text(review)
  prediction = model.predict(preprocessed_input)
  sentiment = 'Positive' if prediction[0][0] > 0.8 else 'Negative'
  return sentiment, prediction[0][0]


In [27]:
example_review = "This movie was fantastic! I loved the plot and story and cast"

sentiment, score = predict_sentiment(example_review)
print(f"Sentiment: {sentiment}, Score: {score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Sentiment: Positive, Score: 0.9783666729927063


In [28]:
example_review = "This movie was okay, one time watch"

sentiment, score = predict_sentiment(example_review)
print(f"Sentiment: {sentiment}, Score: {score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Sentiment: Positive, Score: 0.8926782011985779


In [29]:
example_review = "mehhh, I did not like the plot, It was waste of time"

sentiment, score = predict_sentiment(example_review)
print(f"Sentiment: {sentiment}, Score: {score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Sentiment: Positive, Score: 0.8203023076057434


In [30]:
example_review = "the movie was bad"

sentiment, score = predict_sentiment(example_review)
print(f"Sentiment: {sentiment}, Score: {score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Sentiment: Negative, Score: 0.1937287151813507
